# 🎓 DSPy ReAct Agents - Fundamentos (Abordagem Linear)

**Versão:** 1.0 - Basic Linear  
**Nível:** Iniciante  
**Tempo estimado:** 20-25 minutos  
**Abordagem:** Conceitos → Construção → Execução

---

## 📋 Sobre este Notebook

Este notebook oferece uma **introdução completa e progressiva** aos agentes ReAct em DSPy. Você vai aprender:

✅ O que são agentes ReAct e por que usar DSPy  
✅ Como modelar dados e criar ferramentas para agentes  
✅ Como construir e configurar um agente ReAct do zero  
✅ Como testar e entender o comportamento do agente  
✅ Conceitos fundamentais para otimização (próximo passo)

### 🎯 Pré-requisitos

- Python básico (funções, classes, tipos)
- Conceitos básicos de LLMs (Large Language Models)
- Familiaridade com Jupyter Notebooks
- *Opcional:* Conhecimento de Pydantic ajuda, mas não é essencial

### 📚 Navegação entre Notebooks

**Série DSPy ReAct Agents:**

1. **→ Você está aqui:** Fundamentos (Linear)
2. [Fundamentos (Hands-On)](dspy_agents_basic_handson.ipynb) - Mesmos conceitos, abordagem prática
3. [Otimização Avançada (Linear)](dspy_agents_advanced_linear.ipynb) - Melhorando performance
4. [Otimização Avançada (Hands-On)](dspy_agents_advanced_handson.ipynb) - Otimização na prática

---

## 🚀 Vamos Começar!

Neste notebook, vamos construir um **agente de atendimento ao cliente para uma companhia aérea**. O agente será capaz de:

- 🔍 Buscar voos disponíveis
- ✈️ Reservar passagens aéreas
- 👤 Consultar perfis de usuários
- 🎫 Gerenciar itinerários
- 📝 Abrir tickets de suporte

Vamos construir tudo passo a passo, entendendo cada componente!


---

# 📖 Parte 1: Fundamentos Teóricos

## O que é um Agente ReAct?

**ReAct** significa **Reasoning + Acting** (Raciocínio + Ação). É um padrão arquitetural para agentes de IA que alterna entre:

1. **🤔 Raciocínio (Reasoning):** O agente pensa sobre o que precisa fazer
2. **🔧 Ação (Acting):** O agente executa uma ferramenta/função
3. **📊 Observação:** O agente recebe o resultado da ação
4. **🔄 Repetir:** Continua até completar a tarefa

### Exemplo de Fluxo ReAct:

```
Usuário: "Preciso de um voo de SFO para JFK"
  ↓
Agente pensa: "Preciso buscar voos disponíveis"
  ↓
Agente age: fetch_flight_info("SFO", "JFK", "2025-09-01")
  ↓
Agente observa: [{flight: "AA101", price: 450}, ...]
  ↓
Agente pensa: "Agora tenho as informações, posso responder"
  ↓
Agente responde: "Encontrei 2 voos disponíveis..."
```

## Por que DSPy?

DSPy é um framework que trata **prompts como código**. Benefícios:

- ✅ **Otimização Automática:** DSPy pode melhorar seus agentes automaticamente
- ✅ **Composibilidade:** Componentes reutilizáveis e modulares
- ✅ **Type Safety:** Validação de inputs/outputs com Pydantic
- ✅ **Debugging:** Rastreamento completo de execução
- ✅ **Produção:** Serialização e deployment facilitados

### DSPy vs. Outros Frameworks

| Framework | Abordagem | Otimização |
|-----------|-----------|------------|
| **DSPy** | Programação declarativa | Automática ✅ |
| LangChain | Chains e templates | Manual 🔧 |
| LlamaIndex | RAG-focused | Manual 🔧 |
| AutoGPT | Agentes autônomos | Heurísticas 🤖 |

---


# 🔧 Parte 2: Setup e Configuração

## Instalação de Dependências

Antes de começar, precisamos instalar as bibliotecas necessárias. Execute no terminal:

```bash
pip install dspy-ai python-dotenv pydantic
```

## Imports e Configuração Inicial

Vamos importar todas as bibliotecas que vamos usar. Cada uma tem um propósito específico:


In [ ]:
# Biblioteca principal do DSPy
import dspy

# Para manipulação de datas
from datetime import datetime, timedelta

# Para tipagem e validação de dados
from typing import List, Optional, Dict, Any
from pydantic import BaseModel, Field

# Para manipulação de JSON e geração de IDs únicos
import json
import uuid

# Para carregar variáveis de ambiente (API keys)
from dotenv import load_dotenv
load_dotenv()

print("✅ Todas as bibliotecas importadas com sucesso!")


### 💡 Por que cada biblioteca?

- **dspy**: Framework principal para criar agentes inteligentes
- **pydantic**: Validação de dados e tipagem forte (garante que dados estão corretos)
- **datetime**: Manipular datas de voos
- **typing**: Adicionar tipos aos nossos dados (List, Optional, etc.)
- **json**: Converter objetos Python para formato de texto
- **uuid**: Gerar IDs únicos para reservas e tickets
- **dotenv**: Carregar API keys de forma segura

---

# 🏗️ Parte 3: Modelagem de Dados

## Por que Modelagem de Dados?

Antes de criar o agente, precisamos definir **estruturas de dados** claras. Isso garante que:
- ✅ Dados sejam válidos (não podemos ter voo com preço negativo!)
- ✅ Código seja auto-documentado (sabemos quais campos existem)
- ✅ Erros sejam detectados cedo (antes de chegar ao LLM)

## Modelos Pydantic

Vamos criar modelos para representar:
1. **UserProfile** - Informações do usuário
2. **Flight** - Dados de um voo
3. **Itinerary** - Uma reserva completa
4. **Ticket** - Ticket de suporte


In [ ]:
# Modelo 1: Perfil do Usuário
class UserProfile(BaseModel):
    """Representa o perfil de um cliente da companhia aérea"""
    name: str                                    # Nome do usuário
    user_id: str                                 # ID único no sistema
    email: str                                   # Email para contato
    phone: str                                   # Telefone
    frequent_flyer_number: Optional[str] = None  # Número de viajante frequente (opcional)

# Modelo 2: Voo
class Flight(BaseModel):
    """Representa um voo específico"""
    flight_id: str           # ID único do voo
    flight_number: str       # Número do voo (ex: "AA101")
    departure_airport: str   # Aeroporto de saída (ex: "SFO")
    arrival_airport: str     # Aeroporto de chegada (ex: "JFK")
    departure_time: str      # Hora de saída
    arrival_time: str        # Hora de chegada
    duration_minutes: int    # Duração em minutos
    price: float             # Preço em dólares
    available_seats: int     # Assentos disponíveis

# Modelo 3: Itinerário (Reserva)
class Itinerary(BaseModel):
    """Representa uma reserva de voo"""
    itinerary_id: str           # ID único da reserva
    user_id: str                # ID do usuário que fez a reserva
    flights: List[Flight]       # Lista de voos na reserva
    total_price: float          # Preço total
    booking_date: str           # Data da reserva
    status: str                 # Status: "confirmed", "cancelled", "pending"

# Modelo 4: Ticket de Suporte
class Ticket(BaseModel):
    """Representa um ticket de suporte ao cliente"""
    ticket_id: str                          # ID único do ticket
    user_id: str                            # ID do usuário
    itinerary_id: str                       # ID do itinerário relacionado (se aplicável)
    confirmation_number: str                # Número de confirmação
    issue_description: Optional[str] = None # Descrição do problema
    status: str                             # Status: "active", "resolved", "pending"

print("✅ Modelos de dados criados com sucesso!")
print("\n📊 Temos 4 modelos:")
print("  1. UserProfile - Dados do usuário")
print("  2. Flight - Informações de voo")
print("  3. Itinerary - Reserva completa")
print("  4. Ticket - Suporte ao cliente")


## 🗄️ Banco de Dados Mock

Para este tutorial, vamos usar dados fictícios (mock) em memória. Em produção, você conectaria a um banco de dados real.

Vamos criar:
- **users_db** - Dicionário com usuários
- **flights_db** - Dicionário com voos disponíveis por rota
- **itineraries_db** - Dicionário para armazenar reservas
- **tickets_db** - Dicionário para tickets de suporte


In [ ]:
# Banco de Usuários
users_db = {
    "Adam": UserProfile(
        name="Adam",
        user_id="user_001",
        email="adam@example.com",
        phone="+1-555-0101",
        frequent_flyer_number="FF12345"
    ),
    "Sarah": UserProfile(
        name="Sarah",
        user_id="user_002",
        email="sarah@example.com",
        phone="+1-555-0102"
    )
}

# Banco de Voos (organizados por rota)
flights_db = {
    "SFO-JFK": [
        Flight(
            flight_id="f001",
            flight_number="AA101",
            departure_airport="SFO",
            arrival_airport="JFK",
            departure_time="08:00",
            arrival_time="16:30",
            duration_minutes=330,
            price=450.00,
            available_seats=15
        ),
        Flight(
            flight_id="f002",
            flight_number="UA205",
            departure_airport="SFO",
            arrival_airport="JFK",
            departure_time="14:00",
            arrival_time="22:45",
            duration_minutes=345,
            price=380.00,
            available_seats=8
        )
    ],
    "JFK-LAX": [
        Flight(
            flight_id="f003",
            flight_number="DL302",
            departure_airport="JFK",
            arrival_airport="LAX",
            departure_time="10:00",
            arrival_time="13:30",
            duration_minutes=390,
            price=520.00,
            available_seats=12
        )
    ]
}

# Bancos vazios para armazenar reservas e tickets
itineraries_db = {}
tickets_db = {}

print("✅ Banco de dados mock criado!")
print(f"\n👥 Usuários: {len(users_db)}")
print(f"✈️  Rotas: {len(flights_db)}")
print(f"🎫 Total de voos: {sum(len(flights) for flights in flights_db.values())}")


---

# 🛠️ Parte 4: Criando Ferramentas (Tools)

## O que são Tools?

**Tools (ferramentas)** são funções Python que o agente pode chamar para realizar ações.

**🔑 Pontos importantes:**
- Sempre retorne **string JSON**
- **Docstring** é crucial! O agente lê para decidir quando usar
- Parâmetros devem ter **tipos** claros

Vamos criar 6 ferramentas essenciais para nosso agente de atendimento:


In [ ]:
def fetch_flight_info(departure: str, arrival: str, date: str) -> str:
    """
    Fetch available flights for a specific route and date.
    
    Args:
        departure: Departure airport code (e.g., 'SFO')
        arrival: Arrival airport code (e.g., 'JFK')
        date: Flight date in YYYY-MM-DD format
    
    Returns:
        JSON string with available flights
    """
    route = f"{departure}-{arrival}"
    flights = flights_db.get(route, [])
    
    if not flights:
        return json.dumps({"error": f"No flights found for route {route} on {date}"})
    
    flights_data = [flight.model_dump() for flight in flights]
    return json.dumps({"flights": flights_data, "count": len(flights_data)})

def get_user_info(name: str) -> str:
    """
    Retrieve user profile information.
    
    Args:
        name: User's name
    
    Returns:
        JSON string with user profile
    """
    user = users_db.get(name)
    if not user:
        return json.dumps({"error": f"User {name} not found"})
    
    return json.dumps({"user": user.model_dump()})

def book_flight(user_name: str, flight_id: str, date: str) -> str:
    """
    Book a flight for a user.
    
    Args:
        user_name: Name of the user
        flight_id: ID of the flight to book
        date: Travel date
    
    Returns:
        JSON string with booking confirmation
    """
    user = users_db.get(user_name)
    if not user:
        return json.dumps({"error": f"User {user_name} not found"})
    
    # Find the flight
    flight = None
    for route_flights in flights_db.values():
        for f in route_flights:
            if f.flight_id == flight_id:
                flight = f
                break
        if flight:
            break
    
    if not flight or flight.available_seats <= 0:
        return json.dumps({"error": "Flight not found or no seats available"})
    
    # Create itinerary
    itinerary_id = str(uuid.uuid4())
    confirmation_number = f"CONF{uuid.uuid4().hex[:8].upper()}"
    
    itinerary = Itinerary(
        itinerary_id=itinerary_id,
        user_id=user.user_id,
        flights=[flight],
        total_price=flight.price,
        booking_date=datetime.now().strftime("%Y-%m-%d"),
        status="confirmed"
    )
    
    itineraries_db[itinerary_id] = itinerary
    flight.available_seats -= 1
    
    return json.dumps({
        "success": True,
        "confirmation_number": confirmation_number,
        "itinerary_id": itinerary_id,
        "flight": flight.model_dump(),
        "total_price": flight.price,
        "message": f"Flight {flight.flight_number} booked successfully for {user_name}"
    })

def file_ticket(user_name: str, issue_description: str) -> str:
    """
    File a customer support ticket.
    
    Args:
        user_name: Name of the user filing the ticket
        issue_description: Description of the issue
    
    Returns:
        JSON string with ticket information
    """
    user = users_db.get(user_name)
    if not user:
        return json.dumps({"error": f"User {user_name} not found"})
    
    ticket_id = str(uuid.uuid4())
    ticket = Ticket(
        ticket_id=ticket_id,
        user_id=user.user_id,
        itinerary_id="",
        confirmation_number="",
        issue_description=issue_description,
        status="pending"
    )
    
    tickets_db[ticket_id] = ticket
    
    return json.dumps({
        "success": True,
        "ticket_id": ticket_id,
        "message": f"Support ticket filed successfully. Ticket ID: {ticket_id}"
    })

# Lista de todas as ferramentas
tools = [fetch_flight_info, get_user_info, book_flight, file_ticket]

print("✅ 4 ferramentas criadas com sucesso!")
print("\n🛠️  Ferramentas disponíveis:")
for tool in tools:
    print(f"  • {tool.__name__}")
